In [2]:
from google.colab import drive
drive.mount('/content/drive')
ROOT='''/content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection'''

Mounted at /content/drive


In [4]:
!unzip '''/content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection/Feedback_Obj/static/responses.zip''' -d '''/content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection/Feedback_Obj/static/responses'''

Archive:  /content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection/Feedback_Obj/static/responses.zip
   creating: /content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection/Feedback_Obj/static/responses/responses/
  inflating: /content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection/Feedback_Obj/static/responses/responses/842_e_0.html  
  inflating: /content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection/Feedback_Obj/static/responses/responses/431_a_0.html  
  inflating: /content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection/Feedback_Obj/static/responses/responses/2982_a_0.html  
  inflating: /content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection/Feedback_Obj/static/responses/responses/3248_c_0.html  
  inflating: /content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection/Feedback_Obj/static/responses/responses/494_b_0.html  
  inflating: /content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection/Feedback_Ob

In [1]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

In [6]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_azure','h_detectron','h_google','h_rekognition']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data,dummy_mark='NA'):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    print("[Dummy Test Acc]", dummy_mark)
    return time_used, json.loads(data[-1])["counterbalance"]

def get_gt_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'Human','b':'Machine','c':'Machine','d':'Machine','e':'Machine','f':'Catch'}[pagename.split('_')[1]]

def get_model_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'b':'azure','c':'detectron','d':'google','e':'rekognition'}[pagename.split('_')[1]]

def isRandomStringCatchTrial(pagename):
    wrongWord = {'static/dataset/coco_val_set/2.jpg': 'fire hydrant', 'static/dataset/coco_val_set/5.jpg': 'vase', 'static/dataset/coco_val_set/7.jpg': 'stone', 'static/dataset/coco_val_set/9.jpg': 'door', 'static/dataset/coco_val_set/10.jpg': 'tile floor', 'static/dataset/coco_val_set/11.jpg': 'toilet', 'static/dataset/coco_val_set/12.jpg': 'fruit', 'static/dataset/coco_val_set/15.jpg': 'mountain', 'static/dataset/coco_val_set/16.jpg': 'cloth', 'static/dataset/coco_val_set/17.jpg': 'river', 'static/dataset/coco_val_set/18.jpg': 'skateboard', 'static/dataset/coco_val_set/19.jpg': 'book', 'static/dataset/coco_val_set/23.jpg': 'potted plant', 'static/dataset/coco_val_set/27.jpg': 'toilet', 'static/dataset/coco_val_set/28.jpg': 'laptop', 'static/dataset/coco_val_set/29.jpg': 'curtain', 'static/dataset/coco_val_set/30.jpg': 'bird', 'static/dataset/coco_val_set/32.jpg': 'couch', 'static/dataset/coco_val_set/33.jpg': 'tile floor', 'static/dataset/coco_val_set/34.jpg': 'orange', 'static/dataset/coco_val_set/35.jpg': 'sports ball', 'static/dataset/coco_val_set/36.jpg': 'fork', 'static/dataset/coco_val_set/38.jpg': 'bush', 'static/dataset/coco_val_set/39.jpg': 'sand', 'static/dataset/coco_val_set/42.jpg': 'baseball glove', 'static/dataset/coco_val_set/44.jpg': 'hat', 'static/dataset/coco_val_set/45.jpg': 'skyscraper', 'static/dataset/coco_val_set/46.jpg': 'traffic light', 'static/dataset/coco_val_set/48.jpg': 'stairs', 'static/dataset/coco_val_set/51.jpg': 'car', 'static/dataset/coco_val_set/52.jpg': 'tennis racket', 'static/dataset/coco_val_set/53.jpg': 'sea', 'static/dataset/coco_val_set/54.jpg': 'blender', 'static/dataset/coco_val_set/55.jpg': 'surfboard', 'static/dataset/coco_val_set/57.jpg': 'shoe', 'static/dataset/coco_val_set/58.jpg': 'railroad', 'static/dataset/coco_val_set/60.jpg': 'refrigerator', 'static/dataset/coco_val_set/62.jpg': 'sandwich', 'static/dataset/coco_val_set/63.jpg': 'tent', 'static/dataset/coco_val_set/64.jpg': 'spoon', 'static/dataset/coco_val_set/65.jpg': 'wood floor', 'static/dataset/coco_val_set/66.jpg': 'chair', 'static/dataset/coco_val_set/70.jpg': 'fruit', 'static/dataset/coco_val_set/71.jpg': 'cat', 'static/dataset/coco_val_set/72.jpg': 'desk', 'static/dataset/coco_val_set/73.jpg': 'baseball glove', 'static/dataset/coco_val_set/74.jpg': 'flower', 'static/dataset/coco_val_set/75.jpg': 'skyscraper', 'static/dataset/coco_val_set/76.jpg': 'napkin', 'static/dataset/coco_val_set/77.jpg': 'vase', 'static/dataset/coco_val_set/78.jpg': 'backpack', 'static/dataset/coco_val_set/79.jpg': 'window-blind', 'static/dataset/coco_val_set/80.jpg': 'street sign', 'static/dataset/coco_val_set/81.jpg': 'carrot', 'static/dataset/coco_val_set/82.jpg': 'river', 'static/dataset/coco_val_set/84.jpg': 'window', 'static/dataset/coco_val_set/85.jpg': 'toaster', 'static/dataset/coco_val_set/86.jpg': 'fruit', 'static/dataset/coco_val_set/87.jpg': 'couch', 'static/dataset/coco_val_set/88.jpg': 'person', 'static/dataset/coco_val_set/89.jpg': 'airplane', 'static/dataset/coco_val_set/90.jpg': 'toothbrush', 'static/dataset/coco_val_set/91.jpg': 'motorcycle', 'static/dataset/coco_val_set/92.jpg': 'snowboard', 'static/dataset/coco_val_set/93.jpg': 'bus', 'static/dataset/coco_val_set/94.jpg': 'mirror', 'static/dataset/coco_val_set/95.jpg': 'microwave', 'static/dataset/coco_val_set/96.jpg': 'bear', 'static/dataset/coco_val_set/97.jpg': 'dog', 'static/dataset/coco_val_set/99.jpg': 'fog', 'static/dataset/coco_val_set/100.jpg': 'blender', 'static/dataset/coco_val_set/101.jpg': 'kite', 'static/dataset/coco_val_set/103.jpg': 'clock', 'static/dataset/coco_val_set/104.jpg': 'playingfield', 'static/dataset/coco_val_set/105.jpg': 'cup', 'static/dataset/coco_val_set/111.jpg': 'snowboard', 'static/dataset/coco_val_set/112.jpg': 'branch', 'static/dataset/coco_val_set/113.jpg': 'suitcase', 'static/dataset/coco_val_set/114.jpg': 'knife', 'static/dataset/coco_val_set/115.jpg': 'tennis racket', 'static/dataset/coco_val_set/116.jpg': 'net', 'static/dataset/coco_val_set/117.jpg': 'donut', 'static/dataset/coco_val_set/119.jpg': 'kite', 'static/dataset/coco_val_set/120.jpg': 'leaves', 'static/dataset/coco_val_set/121.jpg': 'window', 'static/dataset/coco_val_set/123.jpg': 'chair', 'static/dataset/coco_val_set/124.jpg': 'tree', 'static/dataset/coco_val_set/126.jpg': 'toaster', 'static/dataset/coco_val_set/127.jpg': 'sheep', 'static/dataset/coco_val_set/128.jpg': 'bush', 'static/dataset/coco_val_set/130.jpg': 'sink', 'static/dataset/coco_val_set/131.jpg': 'truck', 'static/dataset/coco_val_set/132.jpg': 'bowl', 'static/dataset/coco_val_set/133.jpg': 'cat', 'static/dataset/coco_val_set/134.jpg': 'hair drier', 'static/dataset/coco_val_set/136.jpg': 'toaster', 'static/dataset/coco_val_set/137.jpg': 'hair drier', 'static/dataset/coco_val_set/138.jpg': 'road', 'static/dataset/coco_val_set/139.jpg': 'shoe', 'static/dataset/coco_val_set/142.jpg': 'handbag', 'static/dataset/coco_val_set/143.jpg': 'umbrella', 'static/dataset/coco_val_set/144.jpg': 'carrot', 'static/dataset/coco_val_set/145.jpg': 'stop sign', 'static/dataset/coco_val_set/148.jpg': 'cloth', 'static/dataset/coco_val_set/150.jpg': 'dog', 'static/dataset/coco_val_set/152.jpg': 'snow', 'static/dataset/coco_val_set/154.jpg': 'leaves', 'static/dataset/coco_val_set/158.jpg': 'platform', 'static/dataset/coco_val_set/159.jpg': 'shelf', 'static/dataset/coco_val_set/160.jpg': 'snow', 'static/dataset/coco_val_set/162.jpg': 'tv', 'static/dataset/coco_val_set/164.jpg': 'stairs', 'static/dataset/coco_val_set/165.jpg': 'kite', 'static/dataset/coco_val_set/167.jpg': 'car', 'static/dataset/coco_val_set/168.jpg': 'skyscraper', 'static/dataset/coco_val_set/169.jpg': 'cupboard', 'static/dataset/coco_val_set/172.jpg': 'stop sign', 'static/dataset/coco_val_set/173.jpg': 'blanket', 'static/dataset/coco_val_set/174.jpg': 'motorcycle', 'static/dataset/coco_val_set/176.jpg': 'baseball bat', 'static/dataset/coco_val_set/179.jpg': 'book', 'static/dataset/coco_val_set/180.jpg': 'stairs', 'static/dataset/coco_val_set/183.jpg': 'remote', 'static/dataset/coco_val_set/184.jpg': 'dining table', 'static/dataset/coco_val_set/186.jpg': 'laptop', 'static/dataset/coco_val_set/188.jpg': 'carrot', 'static/dataset/coco_val_set/190.jpg': 'bush', 'static/dataset/coco_val_set/191.jpg': 'desk', 'static/dataset/coco_val_set/194.jpg': 'fork', 'static/dataset/coco_val_set/195.jpg': 'sports ball', 'static/dataset/coco_val_set/196.jpg': 'refrigerator', 'static/dataset/coco_val_set/202.jpg': 'carrot', 'static/dataset/coco_val_set/203.jpg': 'tree', 'static/dataset/coco_val_set/204.jpg': 'spoon', 'static/dataset/coco_val_set/205.jpg': 'stop sign', 'static/dataset/coco_val_set/207.jpg': 'table', 'static/dataset/coco_val_set/208.jpg': 'knife', 'static/dataset/coco_val_set/209.jpg': 'plant', 'static/dataset/coco_val_set/210.jpg': 'net', 'static/dataset/coco_val_set/211.jpg': 'knife', 'static/dataset/coco_val_set/212.jpg': 'traffic light', 'static/dataset/coco_val_set/216.jpg': 'sea', 'static/dataset/coco_val_set/217.jpg': 'hot dog', 'static/dataset/coco_val_set/218.jpg': 'apple', 'static/dataset/coco_val_set/219.jpg': 'fog', 'static/dataset/coco_val_set/222.jpg': 'cup', 'static/dataset/coco_val_set/223.jpg': 'cat', 'static/dataset/coco_val_set/224.jpg': 'elephant', 'static/dataset/coco_val_set/225.jpg': 'donut', 'static/dataset/coco_val_set/227.jpg': 'snowboard', 'static/dataset/coco_val_set/229.jpg': 'skateboard', 'static/dataset/coco_val_set/230.jpg': 'zebra', 'static/dataset/coco_val_set/231.jpg': 'cup', 'static/dataset/coco_val_set/233.jpg': 'flower', 'static/dataset/coco_val_set/234.jpg': 'umbrella', 'static/dataset/coco_val_set/235.jpg': 'handbag', 'static/dataset/coco_val_set/236.jpg': 'giraffe', 'static/dataset/coco_val_set/237.jpg': 'airplane', 'static/dataset/coco_val_set/239.jpg': 'apple', 'static/dataset/coco_val_set/240.jpg': 'snowboard', 'static/dataset/coco_val_set/241.jpg': 'cup', 'static/dataset/coco_val_set/242.jpg': 'banana', 'static/dataset/coco_val_set/243.jpg': 'stairs', 'static/dataset/coco_val_set/246.jpg': 'baseball glove', 'static/dataset/coco_val_set/247.jpg': 'house', 'static/dataset/coco_val_set/248.jpg': 'fruit', 'static/dataset/coco_val_set/249.jpg': 'counter', 'static/dataset/coco_val_set/252.jpg': 'motorcycle', 'static/dataset/coco_val_set/253.jpg': 'truck', 'static/dataset/coco_val_set/255.jpg': 'truck', 'static/dataset/coco_val_set/256.jpg': 'sheep', 'static/dataset/coco_val_set/258.jpg': 'elephant', 'static/dataset/coco_val_set/259.jpg': 'road', 'static/dataset/coco_val_set/260.jpg': 'airplane', 'static/dataset/coco_val_set/263.jpg': 'giraffe', 'static/dataset/coco_val_set/264.jpg': 'carrot', 'static/dataset/coco_val_set/265.jpg': 'skis', 'static/dataset/coco_val_set/266.jpg': 'curtain', 'static/dataset/coco_val_set/267.jpg': 'mouse', 'static/dataset/coco_val_set/268.jpg': 'straw', 'static/dataset/coco_val_set/273.jpg': 'cow', 'static/dataset/coco_val_set/275.jpg': 'broccoli', 'static/dataset/coco_val_set/276.jpg': 'bicycle', 'static/dataset/coco_val_set/277.jpg': 'banana', 'static/dataset/coco_val_set/278.jpg': 'table', 'static/dataset/coco_val_set/280.jpg': 'toilet', 'static/dataset/coco_val_set/281.jpg': 'mirror', 'static/dataset/coco_val_set/282.jpg': 'laptop', 'static/dataset/coco_val_set/283.jpg': 'mirror-stuff', 'static/dataset/coco_val_set/284.jpg': 'baseball glove', 'static/dataset/coco_val_set/285.jpg': 'giraffe', 'static/dataset/coco_val_set/286.jpg': 'bear', 'static/dataset/coco_val_set/289.jpg': 'motorcycle', 'static/dataset/coco_val_set/292.jpg': 'surfboard', 'static/dataset/coco_val_set/293.jpg': 'scissors', 'static/dataset/coco_val_set/294.jpg': 'snowboard', 'static/dataset/coco_val_set/295.jpg': 'frisbee', 'static/dataset/coco_val_set/296.jpg': 'plant', 'static/dataset/coco_val_set/297.jpg': 'cell phone', 'static/dataset/coco_val_set/298.jpg': 'refrigerator', 'static/dataset/coco_val_set/302.jpg': 'couch', 'static/dataset/coco_val_set/304.jpg': 'sandwich', 'static/dataset/coco_val_set/305.jpg': 'fire hydrant', 'static/dataset/coco_val_set/306.jpg': 'traffic light', 'static/dataset/coco_val_set/309.jpg': 'backpack', 'static/dataset/coco_val_set/310.jpg': 'frisbee', 'static/dataset/coco_val_set/311.jpg': 'napkin', 'static/dataset/coco_val_set/312.jpg': 'bicycle', 'static/dataset/coco_val_set/314.jpg': 'cell phone', 'static/dataset/coco_val_set/315.jpg': 'stairs', 'static/dataset/coco_val_set/320.jpg': 'house', 'static/dataset/coco_val_set/321.jpg': 'blender', 'static/dataset/coco_val_set/322.jpg': 'stone', 'static/dataset/coco_val_set/324.jpg': 'hot dog', 'static/dataset/coco_val_set/325.jpg': 'bowl', 'static/dataset/coco_val_set/326.jpg': 'flower', 'static/dataset/coco_val_set/327.jpg': 'traffic light', 'static/dataset/coco_val_set/328.jpg': 'carrot', 'static/dataset/coco_val_set/331.jpg': 'orange', 'static/dataset/coco_val_set/335.jpg': 'hair brush', 'static/dataset/coco_val_set/336.jpg': 'skyscraper', 'static/dataset/coco_val_set/338.jpg': 'tent', 'static/dataset/coco_val_set/340.jpg': 'sheep', 'static/dataset/coco_val_set/341.jpg': 'elephant', 'static/dataset/coco_val_set/342.jpg': 'baseball glove', 'static/dataset/coco_val_set/345.jpg': 'toaster', 'static/dataset/coco_val_set/346.jpg': 'window', 'static/dataset/coco_val_set/347.jpg': 'counter', 'static/dataset/coco_val_set/348.jpg': 'fruit', 'static/dataset/coco_val_set/349.jpg': 'couch', 'static/dataset/coco_val_set/350.jpg': 'parking meter', 'static/dataset/coco_val_set/351.jpg': 'hair drier', 'static/dataset/coco_val_set/352.jpg': 'train', 'static/dataset/coco_val_set/354.jpg': 'cake', 'static/dataset/coco_val_set/355.jpg': 'bridge', 'static/dataset/coco_val_set/358.jpg': 'hat', 'static/dataset/coco_val_set/359.jpg': 'grass', 'static/dataset/coco_val_set/360.jpg': 'road', 'static/dataset/coco_val_set/361.jpg': 'umbrella', 'static/dataset/coco_val_set/362.jpg': 'potted plant', 'static/dataset/coco_val_set/365.jpg': 'sandwich', 'static/dataset/coco_val_set/367.jpg': 'napkin', 'static/dataset/coco_val_set/368.jpg': 'hair drier', 'static/dataset/coco_val_set/369.jpg': 'cell phone', 'static/dataset/coco_val_set/370.jpg': 'branch', 'static/dataset/coco_val_set/371.jpg': 'snowboard', 'static/dataset/coco_val_set/373.jpg': 'dog', 'static/dataset/coco_val_set/374.jpg': 'counter', 'static/dataset/coco_val_set/375.jpg': 'truck', 'static/dataset/coco_val_set/376.jpg': 'potted plant', 'static/dataset/coco_val_set/379.jpg': 'umbrella', 'static/dataset/coco_val_set/380.jpg': 'bed', 'static/dataset/coco_val_set/381.jpg': 'pizza', 'static/dataset/coco_val_set/383.jpg': 'pizza', 'static/dataset/coco_val_set/385.jpg': 'bear', 'static/dataset/coco_val_set/386.jpg': 'microwave', 'static/dataset/coco_val_set/387.jpg': 'fork', 'static/dataset/coco_val_set/388.jpg': 'sea', 'static/dataset/coco_val_set/390.jpg': 'car', 'static/dataset/coco_val_set/392.jpg': 'cup', 'static/dataset/coco_val_set/393.jpg': 'cake', 'static/dataset/coco_val_set/394.jpg': 'train', 'static/dataset/coco_val_set/396.jpg': 'pillow', 'static/dataset/coco_val_set/398.jpg': 'sink', 'static/dataset/coco_val_set/405.jpg': 'street sign', 'static/dataset/coco_val_set/407.jpg': 'straw', 'static/dataset/coco_val_set/408.jpg': 'bottle', 'static/dataset/coco_val_set/409.jpg': 'book', 'static/dataset/coco_val_set/410.jpg': 'mountain', 'static/dataset/coco_val_set/411.jpg': 'baseball bat', 'static/dataset/coco_val_set/414.jpg': 'clouds', 'static/dataset/coco_val_set/415.jpg': 'snow', 'static/dataset/coco_val_set/416.jpg': 'cake', 'static/dataset/coco_val_set/417.jpg': 'hair brush', 'static/dataset/coco_val_set/418.jpg': 'book', 'static/dataset/coco_val_set/419.jpg': 'truck', 'static/dataset/coco_val_set/420.jpg': 'knife', 'static/dataset/coco_val_set/423.jpg': 'tv', 'static/dataset/coco_val_set/426.jpg': 'branch', 'static/dataset/coco_val_set/427.jpg': 'teddy bear', 'static/dataset/coco_val_set/430.jpg': 'dining table', 'static/dataset/coco_val_set/431.jpg': 'bush', 'static/dataset/coco_val_set/434.jpg': 'blanket', 'static/dataset/coco_val_set/437.jpg': 'blanket', 'static/dataset/coco_val_set/438.jpg': 'bed', 'static/dataset/coco_val_set/439.jpg': 'snowboard', 'static/dataset/coco_val_set/440.jpg': 'cat', 'static/dataset/coco_val_set/441.jpg': 'baseball glove', 'static/dataset/coco_val_set/442.jpg': 'apple', 'static/dataset/coco_val_set/444.jpg': 'mouse', 'static/dataset/coco_val_set/446.jpg': 'door', 'static/dataset/coco_val_set/448.jpg': 'skateboard', 'static/dataset/coco_val_set/450.jpg': 'book', 'static/dataset/coco_val_set/451.jpg': 'house', 'static/dataset/coco_val_set/452.jpg': 'bicycle', 'static/dataset/coco_val_set/453.jpg': 'fruit', 'static/dataset/coco_val_set/454.jpg': 'baseball bat', 'static/dataset/coco_val_set/455.jpg': 'sink', 'static/dataset/coco_val_set/456.jpg': 'bear', 'static/dataset/coco_val_set/457.jpg': 'rock', 'static/dataset/coco_val_set/458.jpg': 'leaves', 'static/dataset/coco_val_set/459.jpg': 'shoe', 'static/dataset/coco_val_set/460.jpg': 'donut', 'static/dataset/coco_val_set/461.jpg': 'door', 'static/dataset/coco_val_set/462.jpg': 'chair', 'static/dataset/coco_val_set/464.jpg': 'spoon', 'static/dataset/coco_val_set/467.jpg': 'book', 'static/dataset/coco_val_set/470.jpg': 'backpack', 'static/dataset/coco_val_set/471.jpg': 'bridge', 'static/dataset/coco_val_set/472.jpg': 'plant', 'static/dataset/coco_val_set/474.jpg': 'scissors', 'static/dataset/coco_val_set/475.jpg': 'cat', 'static/dataset/coco_val_set/476.jpg': 'stairs', 'static/dataset/coco_val_set/479.jpg': 'dog', 'static/dataset/coco_val_set/480.jpg': 'truck', 'static/dataset/coco_val_set/483.jpg': 'carpet', 'static/dataset/coco_val_set/488.jpg': 'knife', 'static/dataset/coco_val_set/489.jpg': 'fog', 'static/dataset/coco_val_set/491.jpg': 'clock', 'static/dataset/coco_val_set/492.jpg': 'oven', 'static/dataset/coco_val_set/493.jpg': 'parking meter', 'static/dataset/coco_val_set/494.jpg': 'blender', 'static/dataset/coco_val_set/496.jpg': 'eye glasses', 'static/dataset/coco_val_set/497.jpg': 'broccoli', 'static/dataset/coco_val_set/498.jpg': 'clouds', 'static/dataset/coco_val_set/499.jpg': 'branch', 'static/dataset/coco_val_set/501.jpg': 'stairs', 'static/dataset/coco_val_set/503.jpg': 'playingfield', 'static/dataset/coco_val_set/505.jpg': 'frisbee', 'static/dataset/coco_val_set/506.jpg': 'window-blind', 'static/dataset/coco_val_set/507.jpg': 'remote', 'static/dataset/coco_val_set/508.jpg': 'hair drier', 'static/dataset/coco_val_set/509.jpg': 'street sign', 'static/dataset/coco_val_set/510.jpg': 'banner', 'static/dataset/coco_val_set/511.jpg': 'apple', 'static/dataset/coco_val_set/513.jpg': 'tile floor', 'static/dataset/coco_val_set/514.jpg': 'parking meter', 'static/dataset/coco_val_set/515.jpg': 'road', 'static/dataset/coco_val_set/516.jpg': 'bus', 'static/dataset/coco_val_set/518.jpg': 'fire hydrant', 'static/dataset/coco_val_set/520.jpg': 'kite', 'static/dataset/coco_val_set/523.jpg': 'platform', 'static/dataset/coco_val_set/525.jpg': 'pavement', 'static/dataset/coco_val_set/528.jpg': 'hair drier', 'static/dataset/coco_val_set/529.jpg': 'airplane', 'static/dataset/coco_val_set/531.jpg': 'elephant', 'static/dataset/coco_val_set/532.jpg': 'bus', 'static/dataset/coco_val_set/534.jpg': 'straw', 'static/dataset/coco_val_set/536.jpg': 'carpet', 'static/dataset/coco_val_set/537.jpg': 'cell phone', 'static/dataset/coco_val_set/538.jpg': 'keyboard', 'static/dataset/coco_val_set/539.jpg': 'keyboard', 'static/dataset/coco_val_set/541.jpg': 'leaves', 'static/dataset/coco_val_set/543.jpg': 'window', 'static/dataset/coco_val_set/545.jpg': 'playingfield', 'static/dataset/coco_val_set/548.jpg': 'carrot', 'static/dataset/coco_val_set/549.jpg': 'plate', 'static/dataset/coco_val_set/550.jpg': 'couch', 'static/dataset/coco_val_set/552.jpg': 'banner', 'static/dataset/coco_val_set/553.jpg': 'potted plant', 'static/dataset/coco_val_set/554.jpg': 'sink', 'static/dataset/coco_val_set/558.jpg': 'bed', 'static/dataset/coco_val_set/560.jpg': 'horse', 'static/dataset/coco_val_set/561.jpg': 'car', 'static/dataset/coco_val_set/562.jpg': 'truck', 'static/dataset/coco_val_set/564.jpg': 'table', 'static/dataset/coco_val_set/565.jpg': 'shelf', 'static/dataset/coco_val_set/566.jpg': 'counter', 'static/dataset/coco_val_set/568.jpg': 'table', 'static/dataset/coco_val_set/570.jpg': 'cloth', 'static/dataset/coco_val_set/571.jpg': 'straw', 'static/dataset/coco_val_set/572.jpg': 'skateboard', 'static/dataset/coco_val_set/573.jpg': 'truck', 'static/dataset/coco_val_set/574.jpg': 'cloth', 'static/dataset/coco_val_set/575.jpg': 'bridge', 'static/dataset/coco_val_set/578.jpg': 'clock', 'static/dataset/coco_val_set/579.jpg': 'bridge', 'static/dataset/coco_val_set/580.jpg': 'zebra', 'static/dataset/coco_val_set/581.jpg': 'mouse', 'static/dataset/coco_val_set/583.jpg': 'net', 'static/dataset/coco_val_set/584.jpg': 'bus', 'static/dataset/coco_val_set/585.jpg': 'fog', 'static/dataset/coco_val_set/587.jpg': 'sports ball', 'static/dataset/coco_val_set/588.jpg': 'sandwich', 'static/dataset/coco_val_set/590.jpg': 'train', 'static/dataset/coco_val_set/594.jpg': 'keyboard', 'static/dataset/coco_val_set/595.jpg': 'donut', 'static/dataset/coco_val_set/597.jpg': 'sink', 'static/dataset/coco_val_set/599.jpg': 'teddy bear', 'static/dataset/coco_val_set/600.jpg': 'skis', 'static/dataset/coco_val_set/601.jpg': 'vase', 'static/dataset/coco_val_set/602.jpg': 'stone', 'static/dataset/coco_val_set/604.jpg': 'desk', 'static/dataset/coco_val_set/606.jpg': 'clouds', 'static/dataset/coco_val_set/608.jpg': 'street sign', 'static/dataset/coco_val_set/609.jpg': 'banner', 'static/dataset/coco_val_set/612.jpg': 'baseball glove', 'static/dataset/coco_val_set/613.jpg': 'hot dog', 'static/dataset/coco_val_set/616.jpg': 'snow', 'static/dataset/coco_val_set/622.jpg': 'leaves', 'static/dataset/coco_val_set/624.jpg': 'flower', 'static/dataset/coco_val_set/628.jpg': 'carpet', 'static/dataset/coco_val_set/629.jpg': 'napkin', 'static/dataset/coco_val_set/631.jpg': 'pillow', 'static/dataset/coco_val_set/632.jpg': 'window', 'static/dataset/coco_val_set/635.jpg': 'branch', 'static/dataset/coco_val_set/636.jpg': 'banner', 'static/dataset/coco_val_set/637.jpg': 'hat', 'static/dataset/coco_val_set/638.jpg': 'straw', 'static/dataset/coco_val_set/639.jpg': 'baseball glove', 'static/dataset/coco_val_set/641.jpg': 'counter', 'static/dataset/coco_val_set/644.jpg': 'couch', 'static/dataset/coco_val_set/645.jpg': 'house', 'static/dataset/coco_val_set/646.jpg': 'remote', 'static/dataset/coco_val_set/647.jpg': 'cage', 'static/dataset/coco_val_set/648.jpg': 'toothbrush', 'static/dataset/coco_val_set/649.jpg': 'cupboard', 'static/dataset/coco_val_set/650.jpg': 'cell phone', 'static/dataset/coco_val_set/651.jpg': 'window-blind', 'static/dataset/coco_val_set/654.jpg': 'tennis racket', 'static/dataset/coco_val_set/655.jpg': 'cup', 'static/dataset/coco_val_set/656.jpg': 'leaves', 'static/dataset/coco_val_set/657.jpg': 'fire hydrant', 'static/dataset/coco_val_set/661.jpg': 'flower', 'static/dataset/coco_val_set/664.jpg': 'cage', 'static/dataset/coco_val_set/668.jpg': 'handbag', 'static/dataset/coco_val_set/669.jpg': 'book', 'static/dataset/coco_val_set/670.jpg': 'wood floor', 'static/dataset/coco_val_set/672.jpg': 'sea', 'static/dataset/coco_val_set/674.jpg': 'potted plant', 'static/dataset/coco_val_set/676.jpg': 'stairs', 'static/dataset/coco_val_set/677.jpg': 'knife', 'static/dataset/coco_val_set/678.jpg': 'keyboard', 'static/dataset/coco_val_set/681.jpg': 'sports ball', 'static/dataset/coco_val_set/682.jpg': 'knife', 'static/dataset/coco_val_set/683.jpg': 'branch', 'static/dataset/coco_val_set/684.jpg': 'cat', 'static/dataset/coco_val_set/686.jpg': 'carrot', 'static/dataset/coco_val_set/687.jpg': 'frisbee', 'static/dataset/coco_val_set/689.jpg': 'stairs', 'static/dataset/coco_val_set/691.jpg': 'vase', 'static/dataset/coco_val_set/692.jpg': 'horse', 'static/dataset/coco_val_set/693.jpg': 'pillow', 'static/dataset/coco_val_set/694.jpg': 'cow', 'static/dataset/coco_val_set/696.jpg': 'branch', 'static/dataset/coco_val_set/697.jpg': 'suitcase', 'static/dataset/coco_val_set/700.jpg': 'zebra', 'static/dataset/coco_val_set/701.jpg': 'window-blind', 'static/dataset/coco_val_set/702.jpg': 'tree', 'static/dataset/coco_val_set/703.jpg': 'clock', 'static/dataset/coco_val_set/704.jpg': 'door', 'static/dataset/coco_val_set/706.jpg': 'branch', 'static/dataset/coco_val_set/708.jpg': 'clock', 'static/dataset/coco_val_set/714.jpg': 'truck', 'static/dataset/coco_val_set/715.jpg': 'grass', 'static/dataset/coco_val_set/717.jpg': 'mirror-stuff', 'static/dataset/coco_val_set/719.jpg': 'banana', 'static/dataset/coco_val_set/720.jpg': 'toilet', 'static/dataset/coco_val_set/721.jpg': 'fork', 'static/dataset/coco_val_set/722.jpg': 'toaster', 'static/dataset/coco_val_set/723.jpg': 'cupboard', 'static/dataset/coco_val_set/724.jpg': 'fog', 'static/dataset/coco_val_set/725.jpg': 'platform', 'static/dataset/coco_val_set/726.jpg': 'road', 'static/dataset/coco_val_set/727.jpg': 'road', 'static/dataset/coco_val_set/728.jpg': 'grass', 'static/dataset/coco_val_set/730.jpg': 'elephant', 'static/dataset/coco_val_set/731.jpg': 'cat', 'static/dataset/coco_val_set/732.jpg': 'clouds', 'static/dataset/coco_val_set/734.jpg': 'boat', 'static/dataset/coco_val_set/735.jpg': 'tent', 'static/dataset/coco_val_set/736.jpg': 'curtain', 'static/dataset/coco_val_set/737.jpg': 'hair drier', 'static/dataset/coco_val_set/738.jpg': 'window', 'static/dataset/coco_val_set/739.jpg': 'bush', 'static/dataset/coco_val_set/742.jpg': 'napkin', 'static/dataset/coco_val_set/743.jpg': 'door', 'static/dataset/coco_val_set/744.jpg': 'remote', 'static/dataset/coco_val_set/745.jpg': 'hair brush', 'static/dataset/coco_val_set/748.jpg': 'couch', 'static/dataset/coco_val_set/749.jpg': 'tent', 'static/dataset/coco_val_set/752.jpg': 'toaster', 'static/dataset/coco_val_set/754.jpg': 'branch', 'static/dataset/coco_val_set/755.jpg': 'teddy bear', 'static/dataset/coco_val_set/756.jpg': 'laptop', 'static/dataset/coco_val_set/757.jpg': 'hot dog', 'static/dataset/coco_val_set/758.jpg': 'train', 'static/dataset/coco_val_set/760.jpg': 'bottle', 'static/dataset/coco_val_set/761.jpg': 'carpet', 'static/dataset/coco_val_set/762.jpg': 'snowboard', 'static/dataset/coco_val_set/763.jpg': 'flower', 'static/dataset/coco_val_set/764.jpg': 'sandwich', 'static/dataset/coco_val_set/766.jpg': 'dining table', 'static/dataset/coco_val_set/769.jpg': 'hair drier', 'static/dataset/coco_val_set/772.jpg': 'wine glass', 'static/dataset/coco_val_set/774.jpg': 'salad', 'static/dataset/coco_val_set/775.jpg': 'cow', 'static/dataset/coco_val_set/776.jpg': 'blanket', 'static/dataset/coco_val_set/777.jpg': 'blender', 'static/dataset/coco_val_set/780.jpg': 'laptop', 'static/dataset/coco_val_set/783.jpg': 'blender', 'static/dataset/coco_val_set/784.jpg': 'broccoli', 'static/dataset/coco_val_set/785.jpg': 'window', 'static/dataset/coco_val_set/786.jpg': 'toothbrush', 'static/dataset/coco_val_set/787.jpg': 'sea', 'static/dataset/coco_val_set/791.jpg': 'skis', 'static/dataset/coco_val_set/792.jpg': 'branch', 'static/dataset/coco_val_set/793.jpg': 'fruit', 'static/dataset/coco_val_set/794.jpg': 'tree', 'static/dataset/coco_val_set/796.jpg': 'zebra', 'static/dataset/coco_val_set/797.jpg': 'frisbee', 'static/dataset/coco_val_set/801.jpg': 'cow', 'static/dataset/coco_val_set/802.jpg': 'traffic light', 'static/dataset/coco_val_set/803.jpg': 'tennis racket', 'static/dataset/coco_val_set/805.jpg': 'cell phone', 'static/dataset/coco_val_set/807.jpg': 'clouds', 'static/dataset/coco_val_set/808.jpg': 'cupboard', 'static/dataset/coco_val_set/810.jpg': 'clock', 'static/dataset/coco_val_set/811.jpg': 'dining table', 'static/dataset/coco_val_set/812.jpg': 'shoe', 'static/dataset/coco_val_set/813.jpg': 'skateboard', 'static/dataset/coco_val_set/815.jpg': 'airplane', 'static/dataset/coco_val_set/817.jpg': 'mirror-stuff', 'static/dataset/coco_val_set/818.jpg': 'skyscraper', 'static/dataset/coco_val_set/819.jpg': 'branch', 'static/dataset/coco_val_set/822.jpg': 'bird', 'static/dataset/coco_val_set/823.jpg': 'toaster', 'static/dataset/coco_val_set/824.jpg': 'remote', 'static/dataset/coco_val_set/826.jpg': 'fire hydrant', 'static/dataset/coco_val_set/828.jpg': 'book', 'static/dataset/coco_val_set/830.jpg': 'remote', 'static/dataset/coco_val_set/833.jpg': 'window', 'static/dataset/coco_val_set/834.jpg': 'river', 'static/dataset/coco_val_set/835.jpg': 'bowl', 'static/dataset/coco_val_set/838.jpg': 'house', 'static/dataset/coco_val_set/839.jpg': 'suitcase', 'static/dataset/coco_val_set/841.jpg': 'skateboard', 'static/dataset/coco_val_set/842.jpg': 'wood floor', 'static/dataset/coco_val_set/843.jpg': 'keyboard', 'static/dataset/coco_val_set/846.jpg': 'table', 'static/dataset/coco_val_set/849.jpg': 'wine glass', 'static/dataset/coco_val_set/852.jpg': 'rock', 'static/dataset/coco_val_set/854.jpg': 'bench', 'static/dataset/coco_val_set/855.jpg': 'sink', 'static/dataset/coco_val_set/856.jpg': 'handbag', 'static/dataset/coco_val_set/857.jpg': 'platform', 'static/dataset/coco_val_set/858.jpg': 'cup', 'static/dataset/coco_val_set/860.jpg': 'carpet', 'static/dataset/coco_val_set/862.jpg': 'zebra', 'static/dataset/coco_val_set/863.jpg': 'knife', 'static/dataset/coco_val_set/864.jpg': 'vase', 'static/dataset/coco_val_set/865.jpg': 'stop sign', 'static/dataset/coco_val_set/866.jpg': 'door', 'static/dataset/coco_val_set/867.jpg': 'banana', 'static/dataset/coco_val_set/868.jpg': 'tennis racket', 'static/dataset/coco_val_set/870.jpg': 'blender', 'static/dataset/coco_val_set/872.jpg': 'sheep', 'static/dataset/coco_val_set/873.jpg': 'traffic light', 'static/dataset/coco_val_set/874.jpg': 'net', 'static/dataset/coco_val_set/875.jpg': 'plant', 'static/dataset/coco_val_set/877.jpg': 'stairs', 'static/dataset/coco_val_set/878.jpg': 'curtain', 'static/dataset/coco_val_set/882.jpg': 'bird', 'static/dataset/coco_val_set/884.jpg': 'backpack', 'static/dataset/coco_val_set/885.jpg': 'surfboard', 'static/dataset/coco_val_set/887.jpg': 'baseball glove', 'static/dataset/coco_val_set/889.jpg': 'sink', 'static/dataset/coco_val_set/890.jpg': 'zebra', 'static/dataset/coco_val_set/893.jpg': 'oven', 'static/dataset/coco_val_set/894.jpg': 'orange', 'static/dataset/coco_val_set/895.jpg': 'cow', 'static/dataset/coco_val_set/898.jpg': 'snowboard', 'static/dataset/coco_val_set/899.jpg': 'dining table', 'static/dataset/coco_val_set/901.jpg': 'apple', 'static/dataset/coco_val_set/905.jpg': 'truck', 'static/dataset/coco_val_set/906.jpg': 'chair', 'static/dataset/coco_val_set/907.jpg': 'tennis racket', 'static/dataset/coco_val_set/908.jpg': 'potted plant', 'static/dataset/coco_val_set/909.jpg': 'curtain', 'static/dataset/coco_val_set/910.jpg': 'curtain', 'static/dataset/coco_val_set/911.jpg': 'baseball bat', 'static/dataset/coco_val_set/913.jpg': 'window', 'static/dataset/coco_val_set/915.jpg': 'blanket', 'static/dataset/coco_val_set/916.jpg': 'house', 'static/dataset/coco_val_set/917.jpg': 'knife', 'static/dataset/coco_val_set/919.jpg': 'bridge', 'static/dataset/coco_val_set/921.jpg': 'cow', 'static/dataset/coco_val_set/922.jpg': 'toothbrush', 'static/dataset/coco_val_set/923.jpg': 'carrot', 'static/dataset/coco_val_set/924.jpg': 'backpack', 'static/dataset/coco_val_set/925.jpg': 'plate', 'static/dataset/coco_val_set/926.jpg': 'plate', 'static/dataset/coco_val_set/927.jpg': 'teddy bear', 'static/dataset/coco_val_set/928.jpg': 'desk', 'static/dataset/coco_val_set/930.jpg': 'sports ball', 'static/dataset/coco_val_set/931.jpg': 'road', 'static/dataset/coco_val_set/932.jpg': 'tv', 'static/dataset/coco_val_set/935.jpg': 'blender', 'static/dataset/coco_val_set/936.jpg': 'flower', 'static/dataset/coco_val_set/937.jpg': 'pizza', 'static/dataset/coco_val_set/938.jpg': 'umbrella', 'static/dataset/coco_val_set/939.jpg': 'truck', 'static/dataset/coco_val_set/942.jpg': 'parking meter', 'static/dataset/coco_val_set/943.jpg': 'hair brush', 'static/dataset/coco_val_set/944.jpg': 'pillow', 'static/dataset/coco_val_set/945.jpg': 'toothbrush', 'static/dataset/coco_val_set/946.jpg': 'carpet', 'static/dataset/coco_val_set/947.jpg': 'salad', 'static/dataset/coco_val_set/949.jpg': 'playingfield', 'static/dataset/coco_val_set/950.jpg': 'cell phone', 'static/dataset/coco_val_set/951.jpg': 'cage', 'static/dataset/coco_val_set/952.jpg': 'snowboard', 'static/dataset/coco_val_set/954.jpg': 'shelf', 'static/dataset/coco_val_set/957.jpg': 'tree', 'static/dataset/coco_val_set/958.jpg': 'banana', 'static/dataset/coco_val_set/961.jpg': 'platform', 'static/dataset/coco_val_set/962.jpg': 'tree', 'static/dataset/coco_val_set/963.jpg': 'tv', 'static/dataset/coco_val_set/966.jpg': 'banner', 'static/dataset/coco_val_set/968.jpg': 'broccoli', 'static/dataset/coco_val_set/970.jpg': 'book', 'static/dataset/coco_val_set/974.jpg': 'plate', 'static/dataset/coco_val_set/975.jpg': 'tile floor', 'static/dataset/coco_val_set/978.jpg': 'river', 'static/dataset/coco_val_set/980.jpg': 'table', 'static/dataset/coco_val_set/981.jpg': 'straw', 'static/dataset/coco_val_set/982.jpg': 'carpet', 'static/dataset/coco_val_set/987.jpg': 'stairs', 'static/dataset/coco_val_set/989.jpg': 'sandwich', 'static/dataset/coco_val_set/990.jpg': 'salad', 'static/dataset/coco_val_set/991.jpg': 'toothbrush', 'static/dataset/coco_val_set/992.jpg': 'pavement', 'static/dataset/coco_val_set/993.jpg': 'carrot', 'static/dataset/coco_val_set/994.jpg': 'bottle', 'static/dataset/coco_val_set/995.jpg': 'skis', 'static/dataset/coco_val_set/996.jpg': 'hair drier', 'static/dataset/coco_val_set/997.jpg': 'cup', 'static/dataset/coco_val_set/998.jpg': 'cloth', 'static/dataset/coco_val_set/1000.jpg': 'tie', 'static/dataset/coco_val_set/1008.jpg': 'rock', 'static/dataset/coco_val_set/1065.jpg': 'platform', 'static/dataset/coco_val_set/1080.jpg': 'fire hydrant', 'static/dataset/coco_val_set/1094.jpg': 'napkin', 'static/dataset/coco_val_set/1136.jpg': 'plate', 'static/dataset/coco_val_set/1139.jpg': 'cake', 'static/dataset/coco_val_set/1140.jpg': 'traffic light', 'static/dataset/coco_val_set/1159.jpg': 'leaves', 'static/dataset/coco_val_set/1200.jpg': 'bear', 'static/dataset/coco_val_set/1209.jpg': 'napkin', 'static/dataset/coco_val_set/1234.jpg': 'spoon', 'static/dataset/coco_val_set/1249.jpg': 'window', 'static/dataset/coco_val_set/1267.jpg': 'sandwich', 'static/dataset/coco_val_set/1276.jpg': 'toothbrush', 'static/dataset/coco_val_set/1282.jpg': 'frisbee', 'static/dataset/coco_val_set/1284.jpg': 'carpet', 'static/dataset/coco_val_set/1333.jpg': 'tree', 'static/dataset/coco_val_set/1334.jpg': 'flower', 'static/dataset/coco_val_set/1391.jpg': 'tile floor', 'static/dataset/coco_val_set/1395.jpg': 'bed', 'static/dataset/coco_val_set/1425.jpg': 'net', 'static/dataset/coco_val_set/1443.jpg': 'banana', 'static/dataset/coco_val_set/1475.jpg': 'cat', 'static/dataset/coco_val_set/1564.jpg': 'spoon', 'static/dataset/coco_val_set/1621.jpg': 'couch', 'static/dataset/coco_val_set/1634.jpg': 'snow', 'static/dataset/coco_val_set/1650.jpg': 'bench', 'static/dataset/coco_val_set/1652.jpg': 'tie', 'static/dataset/coco_val_set/1669.jpg': 'shelf', 'static/dataset/coco_val_set/1691.jpg': 'sand', 'static/dataset/coco_val_set/1701.jpg': 'plate', 'static/dataset/coco_val_set/1712.jpg': 'desk', 'static/dataset/coco_val_set/1725.jpg': 'mouse', 'static/dataset/coco_val_set/1727.jpg': 'hair brush', 'static/dataset/coco_val_set/1736.jpg': 'sheep', 'static/dataset/coco_val_set/1766.jpg': 'donut', 'static/dataset/coco_val_set/1797.jpg': 'sea', 'static/dataset/coco_val_set/1815.jpg': 'window', 'static/dataset/coco_val_set/1824.jpg': 'street sign', 'static/dataset/coco_val_set/1856.jpg': 'tree', 'static/dataset/coco_val_set/1868.jpg': 'mirror-stuff', 'static/dataset/coco_val_set/1883.jpg': 'carpet', 'static/dataset/coco_val_set/1887.jpg': 'river', 'static/dataset/coco_val_set/1931.jpg': 'parking meter', 'static/dataset/coco_val_set/1936.jpg': 'leaves', 'static/dataset/coco_val_set/1960.jpg': 'zebra', 'static/dataset/coco_val_set/1968.jpg': 'broccoli', 'static/dataset/coco_val_set/2015.jpg': 'donut', 'static/dataset/coco_val_set/2022.jpg': 'window', 'static/dataset/coco_val_set/2127.jpg': 'plant', 'static/dataset/coco_val_set/2131.jpg': 'fruit', 'static/dataset/coco_val_set/2138.jpg': 'road', 'static/dataset/coco_val_set/2155.jpg': 'hair drier', 'static/dataset/coco_val_set/2164.jpg': 'airplane', 'static/dataset/coco_val_set/2169.jpg': 'skateboard', 'static/dataset/coco_val_set/2173.jpg': 'skis', 'static/dataset/coco_val_set/2187.jpg': 'umbrella', 'static/dataset/coco_val_set/2227.jpg': 'shoe', 'static/dataset/coco_val_set/2276.jpg': 'clouds', 'static/dataset/coco_val_set/2278.jpg': 'flower', 'static/dataset/coco_val_set/2312.jpg': 'truck', 'static/dataset/coco_val_set/2337.jpg': 'shoe', 'static/dataset/coco_val_set/2353.jpg': 'cloth', 'static/dataset/coco_val_set/2382.jpg': 'fork', 'static/dataset/coco_val_set/2387.jpg': 'banner', 'static/dataset/coco_val_set/2411.jpg': 'kite', 'static/dataset/coco_val_set/2428.jpg': 'straw', 'static/dataset/coco_val_set/2465.jpg': 'fork', 'static/dataset/coco_val_set/2526.jpg': 'bird', 'static/dataset/coco_val_set/2532.jpg': 'backpack', 'static/dataset/coco_val_set/2546.jpg': 'clouds', 'static/dataset/coco_val_set/2557.jpg': 'street sign', 'static/dataset/coco_val_set/2562.jpg': 'umbrella', 'static/dataset/coco_val_set/2595.jpg': 'bowl', 'static/dataset/coco_val_set/2614.jpg': 'tile floor', 'static/dataset/coco_val_set/2638.jpg': 'window-blind', 'static/dataset/coco_val_set/2642.jpg': 'mouse', 'static/dataset/coco_val_set/2659.jpg': 'sand', 'static/dataset/coco_val_set/2672.jpg': 'cake', 'static/dataset/coco_val_set/2674.jpg': 'cloth', 'static/dataset/coco_val_set/2694.jpg': 'parking meter', 'static/dataset/coco_val_set/2744.jpg': 'bed', 'static/dataset/coco_val_set/2749.jpg': 'umbrella', 'static/dataset/coco_val_set/2753.jpg': 'shoe', 'static/dataset/coco_val_set/2757.jpg': 'suitcase', 'static/dataset/coco_val_set/2810.jpg': 'stone', 'static/dataset/coco_val_set/2818.jpg': 'bus', 'static/dataset/coco_val_set/2860.jpg': 'carrot', 'static/dataset/coco_val_set/2870.jpg': 'cat', 'static/dataset/coco_val_set/2872.jpg': 'bird', 'static/dataset/coco_val_set/2915.jpg': 'hair drier', 'static/dataset/coco_val_set/2959.jpg': 'bush', 'static/dataset/coco_val_set/2970.jpg': 'remote', 'static/dataset/coco_val_set/2982.jpg': 'stone', 'static/dataset/coco_val_set/3003.jpg': 'banana', 'static/dataset/coco_val_set/3020.jpg': 'clouds', 'static/dataset/coco_val_set/3051.jpg': 'tie', 'static/dataset/coco_val_set/3087.jpg': 'pillow', 'static/dataset/coco_val_set/3118.jpg': 'platform', 'static/dataset/coco_val_set/3172.jpg': 'counter', 'static/dataset/coco_val_set/3182.jpg': 'banner', 'static/dataset/coco_val_set/3202.jpg': 'kite', 'static/dataset/coco_val_set/3204.jpg': 'napkin', 'static/dataset/coco_val_set/3248.jpg': 'fork', 'static/dataset/coco_val_set/3258.jpg': 'snow', 'static/dataset/coco_val_set/3266.jpg': 'cloth', 'static/dataset/coco_val_set/3269.jpg': 'stop sign', 'static/dataset/coco_val_set/3272.jpg': 'snowboard', 'static/dataset/coco_val_set/3319.jpg': 'chair', 'static/dataset/coco_val_set/3323.jpg': 'kite', 'static/dataset/coco_val_set/3364.jpg': 'street sign', 'static/dataset/coco_val_set/3366.jpg': 'donut', 'static/dataset/coco_val_set/3374.jpg': 'window-blind', 'static/dataset/coco_val_set/3393.jpg': 'cage', 'static/dataset/coco_val_set/3402.jpg': 'umbrella', 'static/dataset/coco_val_set/3541.jpg': 'apple', 'static/dataset/coco_val_set/3542.jpg': 'cell phone', 'static/dataset/coco_val_set/3555.jpg': 'mirror-stuff', 'static/dataset/coco_val_set/3562.jpg': 'grass', 'static/dataset/coco_val_set/3571.jpg': 'sea', 'static/dataset/coco_val_set/3589.jpg': 'tennis racket', 'static/dataset/coco_val_set/3703.jpg': 'person', 'static/dataset/coco_val_set/3716.jpg': 'bridge', 'static/dataset/coco_val_set/3751.jpg': 'chair', 'static/dataset/coco_val_set/3770.jpg': 'handbag', 'static/dataset/coco_val_set/3805.jpg': 'bed', 'static/dataset/coco_val_set/3808.jpg': 'car', 'static/dataset/coco_val_set/3828.jpg': 'railroad', 'static/dataset/coco_val_set/3855.jpg': 'cloth', 'static/dataset/coco_val_set/3878.jpg': 'grass', 'static/dataset/coco_val_set/3898.jpg': 'car', 'static/dataset/coco_val_set/3917.jpg': 'blanket', 'static/dataset/coco_val_set/3932.jpg': 'snow', 'static/dataset/coco_val_set/3958.jpg': 'spoon', 'static/dataset/coco_val_set/3976.jpg': 'skyscraper', 'static/dataset/coco_val_set/3999.jpg': 'orange', 'static/dataset/coco_val_set/4053.jpg': 'branch', 'static/dataset/coco_val_set/4056.jpg': 'hair drier', 'static/dataset/coco_val_set/4089.jpg': 'fire hydrant', 'static/dataset/coco_val_set/4123.jpg': 'keyboard', 'static/dataset/coco_val_set/4127.jpg': 'river', 'static/dataset/coco_val_set/4158.jpg': 'desk', 'static/dataset/coco_val_set/4212.jpg': 'bear', 'static/dataset/coco_val_set/4250.jpg': 'apple', 'static/dataset/coco_val_set/4316.jpg': 'suitcase', 'static/dataset/coco_val_set/4319.jpg': 'salad', 'static/dataset/coco_val_set/4334.jpg': 'snowboard', 'static/dataset/coco_val_set/4355.jpg': 'sandwich', 'static/dataset/coco_val_set/4358.jpg': 'snow', 'static/dataset/coco_val_set/4374.jpg': 'window-blind', 'static/dataset/coco_val_set/4395.jpg': 'branch', 'static/dataset/coco_val_set/4415.jpg': 'fire hydrant', 'static/dataset/coco_val_set/4418.jpg': 'refrigerator', 'static/dataset/coco_val_set/4441.jpg': 'railroad', 'static/dataset/coco_val_set/4478.jpg': 'playingfield', 'static/dataset/coco_val_set/4487.jpg': 'bicycle', 'static/dataset/coco_val_set/4491.jpg': 'eye glasses', 'static/dataset/coco_val_set/4581.jpg': 'wood floor', 'static/dataset/coco_val_set/4593.jpg': 'person', 'static/dataset/coco_val_set/4621.jpg': 'laptop', 'static/dataset/coco_val_set/4640.jpg': 'couch', 'static/dataset/coco_val_set/4653.jpg': 'bear', 'static/dataset/coco_val_set/4687.jpg': 'fruit', 'static/dataset/coco_val_set/4704.jpg': 'leaves', 'static/dataset/coco_val_set/4707.jpg': 'shelf', 'static/dataset/coco_val_set/4743.jpg': 'refrigerator', 'static/dataset/coco_val_set/4753.jpg': 'stone', 'static/dataset/coco_val_set/4775.jpg': 'plant', 'static/dataset/coco_val_set/4787.jpg': 'road', 'static/dataset/coco_val_set/4798.jpg': 'bed', 'static/dataset/coco_val_set/4838.jpg': 'net', 'static/dataset/coco_val_set/4871.jpg': 'donut', 'static/dataset/coco_val_set/4896.jpg': 'oven', 'static/dataset/coco_val_set/4916.jpg': 'bicycle'}
    wrongWord = wrongWord.values()

    with open(ROOT+"/Feedback_Obj/static/responses/responses/"+pagename[:-1],'r') as f:
        line = f.readlines()[0]
    words = line.split('Speaker Response: ')[-1].split('</h4>')[0].split(', ')
#     print(words)
    if words[0]==words[1]==words[2] and words[0] in wrongWord:
        return False
    else:
        return True

conn = sqlite3.connect(ROOT+"/Feedback_Obj/ec2_db/turing_feedback_obj.db")
cursor = conn.cursor()
cursor.execute("select * from tablechairDB;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'azure':1,'detectron':1,'google':1,'catch':1,'rekognition':1}
oneWord_curr_d = {'0':True,'1':False} # dummy test gt
topic_d = {'1':True,'2':False} # user response to dummy tests
modelname_dict = {'catch':'CatchTrial','azure':'Azure','detectron':'Detectron','rekognition':'Rekognition','google':'Google'}


count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []


confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
dummy_marks_overall = []
for i in range(len(all_data)):
    if all_data[i][15] in [4]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue
        # filter out subjects using dummy trial accuracy
        # original threshold in the paper: 7/12
        dummy_mark = 0
        nTrials = 90
        threshold = nTrials*(7/12) # change to number of total trials
        for dat in data_dict['data']:

            if dat['trialdata']['phase'] == 'TEST':
                present = dat['trialdata']['oneWord_curr'] == '0'
                response_present = dat['trialdata']['topic'] == '1'
                if present == response_present:
                    dummy_mark += 1
        if dummy_mark < threshold:
#             print(dummy_mark)
            continue
        dummy_marks_overall.append(dummy_mark)

        # construct conf matrices
        time_used, counterbalance = print_metadata(all_data[i],"{}/{}".format(dummy_mark,nTrials))
        completed_counterbalances.append(counterbalance)
#         marks = []
#         marks_conf = []
        confmatrices = construct_confmat() # without catch
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))
        marks = []
        marks_conf = []
        participant_count += 1
        nTrial = 0

        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                nTrial+=1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']


#                 trials_dict['Trial_'+str(count)] = trial_data
#                 trial_data = trial_data['trialData']
                trial_data['trialData']['correct response'] = get_gt_from_pagename(trial_data['trialData']['page_name'])
                trial_data['trialData']['choose response'] = trial_data['trialData']['Aclass']

#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    if not isRandomStringCatchTrial(trial_data['trialData']['page_name']):
                        catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                        catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1


                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif nTrial > 40:
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
                        model = get_model_from_pagename(trial_data['trialData']['page_name'])
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                    marks.append(1 if trial_data['trialData']['correct response'] == trial_data['trialData']['choose response'] else 0)
                    marks_conf.append(deepcopy(confmatrices['h_machine']))

#                 trial_data['trialData'] = dat['trialdata']
                if trial_data['trialData']['correct response']  in ['Catch','Swapped'] or nTrial <=40:
                    continue

                count += 1
                trial_data['trialData']['response_gender'] = trial_data['trialData']['Agender']
                del trial_data['trialData']['Agender']

                trial_data['trialData']['response_speaker'] = trial_data['trialData']['choose response'].lower()
                del trial_data['trialData']['choose response']

                trial_data['trialData']['response_object'] = trial_data['trialData']['topic'].lower()
                del trial_data['trialData']['topic']

                trial_data['trialData']['groundtruth'] = trial_data['trialData']['correct response'].lower()


                trial_data['trialData']['object_groundtruth'] = trial_data['trialData']['oneWord_curr'].lower()
                del trial_data['trialData']['oneWord_curr']

                if trial_data['trialData']['correct response'] in ["Machine","Catch","Swapped"]:
                    trial_data['trialData']['machine_groundtruth'] = modelname_dict[model.lower()]
                else:
                    trial_data['trialData']['machine_groundtruth'] = ''

                del trial_data['trialData']['correct response']
#                 trial_data['trialData']["imgtype"] = "naturaldesign"



                trials_dict['Trial_Obj_'+str(count)] = trial_data
#         plot_learning_curve_single_subject(marks_conf,participant_count)
        print(confmatrices)
        print('catch',catch_trial_mat)
#         print('swap',swap_trial_mat)
#     else:
#         print(all_data[i][15])
conn.close()

[Finished Date] 2025-04-02 10:44:07
[Time Used] 18.2 mins
[Questionnaire] {'country': 'United States', 'age': '51', 'engagement': '6', 'difficulty': '6', 'mode': 'live', 'gender': 'male', 'education': 'bachelor', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 85/90
{'h_machine': array([[17.,  3.],
       [ 6., 14.]]), 'h_azure': array([[2., 3.]]), 'h_detectron': array([[1., 4.]]), 'h_google': array([[2., 3.]]), 'h_rekognition': array([[1., 4.]])}
catch [[0. 6.]]
[Finished Date] 2025-04-02 10:44:52
[Time Used] 17.5 mins
[Questionnaire] {'country': 'United States', 'age': '51', 'engagement': '9', 'difficulty': '9', 'mode': 'live', 'gender': 'male', 'education': 'high school', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 80/90
{'h_machine': array([[13.,  7.],
       [ 9., 11.]]), 'h_azure': array([[2., 3.]]), 'h_detectron': array([[3., 2.]]), 'h_google': array([[2., 3.]]), 'h_rekognition': array([[2.

In [ ]:
confmat_overall

{'h_machine': array([[682., 318.],
        [402., 598.]]),
 'h_azure': array([[ 73., 177.]]),
 'h_detectron': array([[141., 109.]]),
 'h_google': array([[103., 147.]]),
 'h_rekognition': array([[ 85., 165.]])}

In [ ]:
catch_trial_mat_overall

array([[ 39., 306.]])

In [7]:
np.mean(dummy_marks_overall)/90

np.float64(0.9233333333333332)

In [8]:
len(trials_dict)/len(dummy_marks_overall)

40.0

In [ ]:
444 /(56.+ 444.)

0.888

In [ ]:
306/(39.+ 306)

0.8869565217391304

In [ ]:
39.+ 306

345.0

In [ ]:
len(trials_dict)

2000

In [ ]:
with open("/home/liuxiao/TuringGithubRunnable/multi_label_prediction/Plot/Data_all/humanjudge_Object_Feedback_testphase.json", 'w') as f:
    json.dump(trials_dict,f)